## Preprocessing and Feature Extraction

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from textblob import TextBlob 

from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk

from argparse import Namespace
from functools import partial
from pathlib import Path
from pprint import pprint

import os
import pickle
import random
import re
import string

## Sentiment Analysis:

Steps taken:
- Converting emojis to text
- We decide to remove all the mentions and hashtagged words, as these will be analysed separately
- Remove Links, as these don't contribute to SA
- Conducting the SA on our preprocessed data

In [2]:
df = pd.read_csv("/Users/harveymiller/Documents/GitHub/text-success/Data/tweets_ukraine_monthly.csv")

Creating a new column so that we can see the adjusted tweet and original versiom

In [ ]:
df.insert(loc=6,
          column='Adjusted Tweet',
          value=df['rendered_content'])

Converting emojis to text

In [ ]:
import emot
import emoji

def demote(text):
    text = emoji.demojize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(demote)

Removing mentions and hashtagged words

In [ ]:
import re

def remove_mentions_hashtags(text):
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_mentions_hashtags)

Removing links

In [ ]:
import re

def remove_links(text):
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'bit.ly/\S+', '', text) # remove bitly links
    text = text.strip('[link]') # remove [links]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_links)

Insert Polarity Score Column

In [ ]:
df.insert(loc=7,
          column='Polarity Score',
          value=df['Adjusted Tweet'])

Sentiment Analysis using NLTK's VADER

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def sentiment_analysis(text):  
    text = sia.polarity_scores(text)
    return text

df['Polarity Score'] = df['Polarity Score'].apply(sentiment_analysis)
    

Creating columns for:
- Negative Score
- Neutral Score
- Positive Score
- Compound Score [-1,1]

In [ ]:
df.insert(loc=8,
          column='Negative Score',
          value=df['Polarity Score'])

df.insert(loc=9,
          column='Neutral Score',
          value=df['Polarity Score'])

df.insert(loc=10,
          column='Positive Score',
          value=df['Polarity Score'])

df.insert(loc=11,
          column='Compound Score',
          value=df['Polarity Score'])

In [ ]:
def negative_score(text):
    text = text['neg']
    return text

df['Negative Score'] = df['Negative Score'].apply(negative_score)

In [ ]:
def neutral_score(text):
    text = text['neu']
    return text

df['Neutral Score'] = df['Neutral Score'].apply(neutral_score)

In [ ]:
def positive_score(text):
    text = text['pos']
    return text

df['Positive Score'] = df['Positive Score'].apply(positive_score)

In [ ]:
def compound_score(text):
    text = text['compound']
    return text

df['Compound Score'] = df['Compound Score'].apply(compound_score)

In [ ]:
df.head()

Sentiment Analysis using TextBlob

In [ ]:
df.insert(loc=12,
          column='Polarity Score_textblob',
          value=df['Adjusted Tweet'])

In [ ]:
df.insert(loc=13,
          column='Subjectivity Score_textblob',
          value=df['Adjusted Tweet'])

In [ ]:
from textblob import TextBlob

#Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

df['Polarity Score_textblob'] = df['Polarity Score_textblob'].apply(getPolarity)
df['Subjectivity Score_textblob'] = df['Subjectivity Score_textblob'].apply(getSubjectivity)


In [ ]:
df.head()

## Further manipulating the tweet

Steps taken:
- Lowercase
- Punctuation
- Tokenization
- Stopword filtering
- Stemming

Changing all text to lowercase

In [ ]:
def lowercase(text):    
    text = text.lower()
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(lowercase)

Removing all Punctuation

In [ ]:
import string

def punctuation_remove(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(punctuation_remove)

Tokenizing

In [ ]:
from nltk import word_tokenize

def tokenize(text):
    text = word_tokenize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(tokenize)

Stopword Filtering

In [ ]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stop_words]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_stopwords)

Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stem(text):
    stemmed = [porter.stem(word) for word in text]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(stem)

In [ ]:
df.head()

## Time of Day

In [3]:
#create a column for hours:
df.insert(loc=2,
          column='hour',
          value=df['date'])

In [4]:
from datetime import *

#return the hour of the tweet
def hour(text):
    text = datetime.strptime(text, "%Y-%m-%d %H:%M:%S+00:00")
    hour = str(text.time())[0:2]
    return int(hour)

df['hour'] = df['hour'].apply(hour)

In [5]:
#insert column for the time of day
df.insert(loc=3,
          column='time of day',
          value=df['hour'])

In [6]:
def time_of_day(text):  
    if ((text < 4) & (text > 8 )):
        text == 'Early Morning'
    elif ((text < 8) & (text > 12 )):
        text == 'Morning'
    elif ((text < 12) & (text > 16 )):
        text == 'Noon'
    elif ((text < 16) & (text > 20 )):
        text == 'Eve'
    elif ((text < 20) & (text > 24 )):
        text == 'Night'
    elif ((text < 24) & (text > 4 )):
        text == 'Late Night'
    return text

df['time of day'] = df['time of day'].apply(time_of_day)


In [7]:
df.head()

,id,date,hour,time of day,user,user_followers,raw_content,rendered_content,likes,retweets,replies,quoteCount,hashtags,lang,media,mentionedUsers
0,1486661338390831105,2022-01-27 11:24:18+00:00,11,11,WorldToBe1,10769,#Russia-#Ukraine debate sparks fiery exchange ...,#Russia-#Ukraine debate sparks fiery exchange ...,0,0,0,0,"['Russia', 'Ukraine', 'CNN', 'USA', 'EU', 'NAT...",en,NaN,NaN
1,1486105614803775490,2022-01-25 22:36:03+00:00,22,22,embeegle,2246,Can you say pipeline? A larger cut coming to ...,Can you say pipeline? A larger cut coming to ...,1,1,0,0,"['PutinsPuppet', 'ukrainewar']",en,NaN,NaN
2,1486056169013661697,2022-01-25 19:19:34+00:00,19,19,Vlad_Mykhnenko,1068,Foreign exchange markets somehow are not betti...,Foreign exchange markets somehow are not betti...,1,0,0,0,"['russianinvasion', 'ukrainewar']",en,[Photo(previewUrl='https://pbs.twimg.com/media...,NaN
3,1486019310069989376,2022-01-25 16:53:07+00:00,16,16,RaVe_74,5309,#borisjohnson's expertise in foreign affairs -...,#borisjohnson's expertise in foreign affairs -...,3,1,0,0,"['borisjohnson', 'freenazanin', 'BrexitShamble...",en,NaN,NaN
4,1485989417084985347,2022-01-25 14:54:20+00:00,14,14,miamicool,1035,"Seems that #ukrainewar just became that ""line ...","Seems that #ukrainewar just became that ""line ...",0,0,0,0,['ukrainewar'],en,NaN,NaN
